# Basic Optickle Modeling of a Fabry Perot Cavity

This notebook makes a simple Fabry Perot cavity in Optickle and computes the frequency response and response to sweeping drives. State space and zpk representations of the optomechanical plant are computed at the end.

[__1.__](#model) Model definition

[__2.__](#frequency-response) Compute the frequency response

[__3.__](#sweep) Compute the response to sweeping drives

[__4.__](#ss-zpk) Find state space and zpk representations of the optomechanical plant

The BasicFinesseFP notebook goes through the identical calculations with Finesse.

#### Python MATLAB engine
The Python MATLAB engine needs to be initialized before using Optickle. This takes a few seconds but only needs to be done once.

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [ ]:
import numpy as np
import qlance.optickle as pyt
from qlance.plotting import plotTF
from qlance.controls import DegreeOfFreedom
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.constants as scc
%matplotlib inline

In [ ]:
mpl.rc('figure', figsize=(12, 9))

mpl.rcParams.update({'text.usetex': False,
                     'mathtext.fontset': 'cm',
                     'lines.linewidth': 3,
                     'lines.markersize': 10,
                     'font.size': 16,
                     'axes.grid': True,
                     'grid.alpha': 0.5,
                     'legend.loc': 'best',
                     'savefig.dpi': 80,
                     'pdf.compression': 9})

#### Adding the Optickle path

The path to Optickle needs to be added to the MATLAB engine. If the path variable `OPTICKLE_PATH` is set, this can be done with
```python
pyt.addOpticklePath(eng)
```
If the variable is not set, this is done with
```python
pyt.addOpticklePath(eng, optickle_path)
```
where `optickle_path` is a string that specifies the path to Optickle.

In [ ]:
pyt.addOpticklePath(eng)

<a name="model"> </a>

## Model Definition

Optickle models can be built directly in Python using the same commands as are used to build models in MATLAB, with a few added conveniences. Optickle models defined in a MATLAB script can also be loaded directly into QLANCE.

Models are started with
```python
opt = pyt.Optickle(eng, opt_name, *args, **kwargs)
```
where `eng` is the MATLAB engine initialized above and `opt_name` is a string giving the name of the model. *If multiple models are defined, they must have different names.*

With Optickle you must specify which sidebands to compute when initializing the model. This is done by specifying a vector of RF frequencies relative to the carrier and passing it as an argument when defining the model. So
```python
vRF = np.array([-11e6, 0, 11e6])
opt = pyt.Optickle(eng, 'opt', vRF=vRF)
```
makes a model called `'opt'` that will track the carrier and the +11 MHz and -11 MHz sidebands. If `vRF` is not specified, only the carrier is tracked.

One convenience in QLANCE not present when building models in MATLAB is that components can be added with non-default parameters just by specifying a keyword rather than listing all the parameters. Most functions have doc strings, so the usage and default parameters can be found easily using, for example, `help(opt.addMirror)`.

In [ ]:
# define some parameters
fmod = 11e6  # modulation frequency for PDH sensing [Hz]
gmod = 0.1   # modulation depth
Pin = 1      # input power [W]
Ti = 0.014   # input coupler transmissivity
Lcav = 40e3  # cavity length [m]
mass = 470   # mirror mass [kg]

# track the carrier and first order sidebands
vRF = np.array([-fmod, 0, fmod])

# start a model named 'opt'
opt = pyt.Optickle(eng, 'opt', vRF)

# make the cavity
opt.addMirror('EX')          # add a perfectly reflecting mirror
opt.addMirror('IX', Thr=Ti)  # add a mirror with transmissivity Ti
opt.addLink('IX', 'fr', 'EX', 'fr', Lcav)
opt.addLink('EX', 'fr', 'IX', 'fr', Lcav)

# set mechanical response
for optic in ['EX', 'IX']:
    opt.setMechTF('EX', [], [0, 0], 1/mass)

# add input
opt.addSource('Laser', np.sqrt(Pin)*(vRF == 0))
opt.addModulator('AM', 1)   # amplitude modulator
opt.addModulator('PM', 1j)  # phase modulator
opt.addRFmodulator('Mod', fmod, 1j*gmod)  # RF modulator for PDH sensing
opt.addLink('Laser', 'out', 'AM', 'in', 0)
opt.addLink('AM', 'out', 'PM', 'in', 0)
opt.addLink('PM', 'out', 'Mod', 'in', 0)
opt.addLink('Mod', 'out', 'IX', 'bk', 0)

# add DC and RF photodiodes
opt.addSink('REFL')
opt.addLink('IX', 'bk', 'REFL', 'in', 0)
opt.addReadout('REFL', fmod, 5)

Note that it is the amplitude rather than the power that is given when adding a laser with `addSource` and that you must specify the amplitude of all field components. Since `vRF == 0` is an array that is 0 everywhere except where `vRF` is zero,
```python
opt.addSource('Laser', np.sqrt(Pin)*(vRF == 0))
```
adds a laser, called `'Laser'`, with power `Pin` in the carrier and with no power in the sidebands.

<p>&nbsp;</p>

The command
```python
opt.setMechTF(optic, zs, ps, k)
```
sets the mechanical response of a mirror to a mechanical plant specified by zeros, poles, and a gain and is described in more detail in the torsional spring example. In this example we treat the mirrors as free masses (which have transfer functions $1/ms^2$). It is not necessary to set the mechanical response for optics if radiation pressure effects are not needed. We set it here so that we can compute the response to laser amplitude modulation.

<p>&nbsp;</p>

`addReadout` is a convenience function that adds RF and DC probes to a detection port. So the last command above
```python
opt.addReadout('REFL', fmod, 5)
```
added three probes to the `REFL` sink:
 1. A DC photodiode named `REFL_DC`
 2. An RF photodiode named `REFL_I` demodulated at frequency `fmod` with phase 5.
 3. An RF photodiode named `REFL_Q` demodulated at frequency `fmod` with phase 5 + 90 = 95.

<a name="frequency-response"> </a>

## Frequency Response

Transfer functions are calculated by specifying a frequency vector `ff` at which to calculate the response
```python
opt.run(ff)
```
By default this calculates
1. The AC transfer functions from drives to probes, i.e. the optomechanical plant.
2. The radiation pressure modifications to the mechanical response of the drives, i.e. the "radiation pressure loop suppression function". This is explained in the torsional spring example and in detail in the control system example, but we do not need it here.
3. The quantum noise from each photodiode. In more complicated models this can take longer to compute and can be ignored with the `noise=False` keyword to speed up the calculations. We do not need quantum noise here.

In [ ]:
# compute the AC response matrix, i.e. the optical plant
fmin = 1e-1
fmax = 10e3
npts = 1000
ff = np.logspace(np.log10(fmin), np.log10(fmax), npts)
opt.run(ff, noise=False)  # ignore quantum noise

After running the model, transfer functions from any drives `drives` to any probes `probes` can be calculated with
```python
opt.getTF(probes, drives)
```
The variables `probes` and `drives` can be strings specifying the probes and drives or they can be dictionaries specifying linear combinations of probes and drives. For example,
```python
opt.getTF('AS_Q', 'EX')
```
computes the respone at the probe `AS_Q` to motion of the mirror `EX`, while
```python
opt.getTF('AS_Q', {'EX': 1/2, 'EY': -1/2})
```
computes the response at the probe `AS_Q` to the differentional motion of the mirrors `EX` and `EY` moving 180 degrees out of phase.

<p>&nbsp;</p>

The convenience function `plotTF` directly plots transfer functions. The optional third and fourth arguments are existing magnitude and phase axes so that multiple functions can be plotted on the same plot. Note that using the python `*args` shortcut the following two are equivalent
```python
plotTF(probes, drives, fig.axes[0], fig.axes[1])
plotTF(probes, drives, *fig.axes)
```

In [ ]:
fig = opt.plotTF('REFL_I', 'EX', label='REFL_I')
opt.plotTF('REFL_Q', 'EX', fig.axes[0], fig.axes[1], ls='-.', label='REFL_Q');
fig.axes[0].legend()
fig.axes[0].set_title('Response to EX Motion')
fig.set_size_inches((8, 11));

### Other types of frequency response: laser frequency, phase, intensity, and amplitude modulation

In addition to the position transfer functions above, QLANCE also calculates angular transfer functions (pitch and yaw) as well as laser frequency, phase, intensity, and amplitude modulation. The torsional spring example gives an example of pitch response.

Transfer functions are computed as before with an additional `doftype` argument:
```python
opt.getTF('REFL_I', 'PM', doftype='drive')   # laser phase modulation
opt.getTF('REFL_I', 'AM', doftype='drive')   # laser amplitude modulation
opt.getTF('REFL_I', 'Mod', doftype='phase')  # oscillator phase noise
opt.getTF('REFL_I', 'Mod', doftype='amp')    # oscillator amplitude noise
```
help(opt.getTF) lists all of the supported transfer functions.

<p>&nbsp;</p>

Optickle calculates laser phase and amplitude modulation. (Note that Finesse computes laser frequency and intensity modulation.) Since frequency is the time derivative of phase, phase modulation is converted to frequency modulation by dividing by $\mathrm{i}f$. Since intensity is the square of amplitude, amplitude modulation is converted to intensity modulation by dividing by 2.

In [ ]:
tf_phase = opt.getTF('REFL_I', 'PM', doftype='drive')  # phase response at REFL_I
tf_amp = opt.getTF('REFL_I', 'AM', doftype='drive')    # amplitude response at REFL_I

In [ ]:
fig = plotTF(ff, tf_phase/(1j*ff), label='Frequency [W/Hz]');
plotTF(ff, tf_phase, *fig.axes, label='Phase [W/rad]');
fig.axes[0].legend()
fig.axes[0].set_ylim(1e-5, 1e0);
fig.set_size_inches((8, 11));

In [ ]:
fig = opt.plotTF('REFL_I', 'AM', doftype='drive');
fig.axes[0].set_title('Laser Amplitude Response');
fig.set_size_inches((8, 11));

### Using DegreeOfFreedom

Transfer functions can also be computed with `DegreeOfFreedom` instances which are used in control systems. These classes store the drives and doftype of a degree of freedom and can optionally store the probes used to detect them as well.

For example all of the following compute the same transfer function
```python
PM = DegreeOfFreedom('AM', doftype='drive', probes='REFL_I')
opt.getTF('REFL_I', 'AM', doftype='drive')
opt.getTF('REFL_I', AM)
opt.getTF(AM)
```

In [ ]:
AM1 = DegreeOfFreedom('AM', doftype='drive')  # probes are optional if specified when calculating the TF
AM2 = DegreeOfFreedom('AM', probes='REFL_I', doftype='drive')
fig = opt.plotTF('REFL_I', 'AM', doftype='drive');
opt.plotTF('REFL_I', AM1, *fig.axes, ls='-.')
opt.plotTF(AM2, *fig.axes, ls=':')
fig.axes[0].set_title('Laser Amplitude Response');
fig.set_size_inches((8, 11));

#### Saving and loading optomechanical plants and radiation pressure modifications for future use

Note also that the results of a simulation can be exported to an hdf5 file and loaded for future analysis. This is useful in complex models that take a long time to run since they only need to be calculated once and can then be analyzed in the future without having to do the calculations again. Since no simulations are needed for previously run data, Optickle and MATLAB do not need to be installed to analyze previously run data. The exported hdf5 file can then be compared with a Finesse simulation by a user who does not have Optickle or MATLAB. Similarly, simulations run with Finesse can be exported and comparred with an Optickle simulation by a user who does not have Finesse.

For example, to save a model to the file `'pdh_freq_resp.hdf5'`
```python
opt.save('pdh_freq_resp.hdf5')
```
and to load it back in a future script
```python
import qlance.plant as plant
opt = plant.OpticklePlant()
opt.load('pdh_freq_resp.hdf5')
```
All analysis functions can be done on this new `opt` object created from a previously computed optomechanical plant, but the underlying Finesse Optickle model does not exist anymore so no further Optickle simulations can be run with it.

To use a model calculated by Finesse
```python
katFR = plant.FinessePlant()
katFR.load('pdh_freq_resp_finesse.hdf5')
```
The analysis functions on this `katFR` object are almost identical to those of the `opt` object and can be analyzed without Finesse installed.

<a name="sweep"> </a>

## Sweeping Drives

To compute the response of a model to sweeping drives
```python
opt.sweepLinear(spos, epos, npts)
```
where `spos` and `epos` are dictionaries of the starting and ending positions of the drives to sweep and `npts` is the number of points to compute. For example
```python
xf = {'EX': 5e-9}
xi = {'EX': -5e-9}
opt.sweepLinear(xi, xf, 100)
```
sweeps the drive `EX` from -5 nm to +5 nm and
```python
xf = dict(EX=5e-9, EY=-5e-9)
xi = {k: -v for k, v in xf.items()}  # in this case equivalent to dict(EX=-5e-9, EY=5e-9)
opt.sweepLinear(xi, xf, 100)
```
sweeps `EX` from -5 nm to +5 nm while sweeping `EY` from +5 nm to -5 nm.

In [ ]:
# sweep from -5 nm to 5 nm
xf = {'EX': 5e-9}   # final position
xi = {'EX': -5e-9}  # initial position [m]
opt.sweepLinear(xi, xf, 1000)

After running the model, the sweep signals are computed with
```python
poses, sig = opt.getTF(probe, drive)
```
This returns the signal `sig` as measured by `probe` as a function of the drive `drive` positions `poses`.

In [ ]:
# get the error signals in REFL_I and REFL_Q
poses, sweepI = opt.getSweepSignal('REFL_I', 'EX')
_, sweepQ = opt.getSweepSignal('REFL_Q', 'EX')

In [ ]:
# compute the slope of the error signals
nn = int(len(poses)/2)  # index of the center x0 of the sweep
nh = nn + 2             # index of x0 + dx
nl = nn - 2             # index of x0 - dx
dx = poses[nh] - poses[nl]
dI = (sweepI[nh] - sweepI[nl]) / dx
dQ = (sweepQ[nh] - sweepQ[nl]) / dx

`plotSweepSignal` is another convenience function, like `plotTF`, which plots sweeps directly. The optional fourth argument is an existing figure so that multiple signals can be plotted on the same plot.

In [ ]:
fig = opt.plotSweepSignal('REFL_I', 'EX', label='REFL_I')
opt.plotSweepSignal('REFL_Q', 'EX', fig, label='REFL_Q')
ax = fig.gca()

# plot the error signal slopes
ymin, ymax = ax.get_ylim()
ax.plot(poses, poses*dI, 'C3:', label='dI/dx');
ax.plot(poses, poses*dQ, 'C2:', label='dQ/dx');
ax.set_ylim(ymin, ymax)

ax.legend()
ax.set_xlabel('EX position [m]')
ax.set_ylabel('Power [W]');